In [ ]:
from torch.utils.data import Dataset
from datasets import load_from_disk
from transformers import BertModel
import torch

class BizDataset(Dataset):
    def __init__(self,split):
        self.dataset = load_from_disk(r"D:\Workspace\regression\huggingface\dataset")
        if split=="train":
            self.dataset=self.dataset["train"]
        elif split=="test":
            self.dataset=self.dataset["test"]
        elif split=="validation":
            self.dataset=self.dataset["validation"]
        else:
            print("split is wrong")
           
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,item):
        text = self.dataset[item]["text"]
        label = self.dataset[item]["label"]
        return text,label
    
DEVICE= torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_name="bert-base-chinese"
model_name=r"D:\Workspace\regression\huggingface\model\bert-base-chinese\models--bert-base-chinese\snapshots\c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f"

pretrained = BertModel.from_pretrained(model_name).to(DEVICE)

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768,2)
    # 前向推理运算
    def forward(self,input_ids,attention_mask,token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,attention_mask= attention_mask,token_type_ids=token_type_ids)
        
        out= self.fc(out.last_hidden_state[:,0])
        out = out.softmax(dim=1)
        return out

In [ ]:
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

test_ds= BizDataset("test")
test_loader = DataLoader(test_ds,batch_size=16,shuffle=False)
model=Model().to(DEVICE)

model.eval()
predictions,true_labels=[],[]

with torch.no_grad():
    for batch in test_loader:
        outputs = model(input_ids =batch["input_ids"] ,attention_mask= batch["attention_mask"],token_type_ids=batch["token_type_ids"])
        _,preds = torch.max(outputs,dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch["labels"].cpu().numpy())

accuracy = accuracy_score(true_labels,predictions)
print(f"Accuracy:{accuracy:.4f}")

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score
precision = precision_score(true_labels,predictions,average='weighted')
print(f"Accuracy:{accuracy:.4f}")
recall = recall_score(true_labels,predictions,average='weighted')
print(f"Recall:{recall:.4f}")
f1_score = f1_score(true_labels,predictions,average='weighted')
print(f"F1Score:{f1_score:.4f}")